# Project I - Image Classification

**Team**: Filip Kołodziejczyk, Jerzy Kraszewski

## Introduction

The goal of this project is to create a model that can classify images of 10 different classes. The dataset used for this project is the CINIC-10 dataset, which is a combination of CIFAR-10 and ImageNet. The dataset contains 270,000 images, which are divided into 10 classes of equal size. The classes are: airplane, automobile, bird, cat, deer, dog, frog, horse, ship, and truck. The images are 32x32 pixels in size and are in RGB format. Data is divided into training, validation, and test sets, equally for each class.
More details about the dataset can be found [here](https://datashare.ed.ac.uk/handle/10283/3192) and [here](https://www.kaggle.com/datasets/mengcius/cinic10/data).

TODO: Add citation for this dataset

## Environment setup

We load all the necessary libraries and set an appropriate backend for the PyTorch for most optimal performance.

In [21]:
import os
import shutil
import time
from zipfile import ZipFile

import pandas as pd
import timm
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision as tv
from IPython.display import display
from torch.utils.data import DataLoader, Subset, default_collate
from torchvision.transforms import v2 as T
from tqdm import tqdm

if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

colab = "COLAB_GPU" in os.environ
if colab:
    from google.colab import drive

    drive.mount("/content/drive")

print(f"Using device: {device}")

Using device: mps


## Extracting and adjusting data split

The original dataset has predefined split of data. We adjust it there.

In [4]:
# Dataset must be downloaded from the link provided in Introduction and put into `data` directory.
# It should be renamed to `cinic10.zip`.

archive_path = "data/cinic10.zip" if not colab else "/content/drive/MyDrive/cinic10.zip"
data_dir = "data/cinic10" if not colab else "/content/cinic10"
data_subdirs = ["train", "test", "valid"]
props = [0.7, 0.15, 0.15]  # Train, test, valid proportions

if sum(props) != 1:
    raise ValueError("Props must sum to 1")

# Extracting the data
with ZipFile(archive_path, "r") as zip_ref:
    zip_ref.extractall(data_dir)

classes = os.listdir(os.path.join(data_dir, "train"))
num_classes = len(classes)

# Changing the data split
for cls in classes:
    dirs = [os.path.join(data_dir, subdir, cls) for subdir in data_subdirs]
    sizes = [len(os.listdir(d)) for d in dirs]
    total = sum(sizes)
    target_sizes = [int(p * total) for p in props]
    diffs = [target_sizes[i] - sizes[i] for i in range(len(sizes))]

    for i in range(len(diffs)):
        if diffs[i] < 0:
            for j in range(len(diffs)):
                if diffs[j] > 0:
                    count = min(abs(diffs[i]), diffs[j])
                    files = os.listdir(dirs[i])
                    files = files[:count]
                    for f in files:
                        shutil.move(os.path.join(dirs[i], f), os.path.join(dirs[j], f))
                    diffs[i] += count
                    diffs[j] -= count

# Checking the sizes
cls_sizes = {}
for cls in classes:
    cls_sizes[cls] = [
        len(os.listdir(os.path.join(data_dir, subdir, cls))) for subdir in data_subdirs
    ]
pd.DataFrame.from_dict(
    cls_sizes, orient="index", columns=[f"{set} size" for set in data_subdirs]
)

,train size,test size,valid size
frog,18900,4050,4050
automobile,18900,4050,4050
airplane,18900,4050,4050
bird,18900,4050,4050
ship,18900,4050,4050
truck,18900,4050,4050
dog,18900,4050,4050
horse,18900,4050,4050
cat,18900,4050,4050
deer,18900,4050,4050


## Loading the data

In [22]:
transforms_no_aug = T.Compose(
    [
        T.PILToTensor(),
        T.Resize((224, 224)),  # Default input size for most models
        T.ToDtype(torch.float32, scale=True),
        T.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        ),  # Default values for ImageNet
    ]
)

train_path = os.path.join(data_dir, "train")
test_path = os.path.join(data_dir, "test")
valid_path = os.path.join(data_dir, "valid")


def get_data(
    transforms: T.Compose, collate_fn=None, batch_size: int = 256
) -> tuple[DataLoader, DataLoader, DataLoader]:
    """
    Returns the data loaders of train, test and validation sets.

    Args:
    - tranforms: Transformations to be applied to the train set (augmentations).
    - batch_size: Batch size to be used for the data loaders (test and validation sets have double the size).

    Returns:
    - train_loader: DataLoader of the train set.
    - test_loader: DataLoader of the test set.
    - valid_loader: DataLoader of the validation set.
    """

    train = tv.datasets.ImageFolder(train_path, transform=transforms)
    test = tv.datasets.ImageFolder(test_path, transform=transforms_no_aug)
    valid = tv.datasets.ImageFolder(valid_path, transform=transforms_no_aug)

    # TODO: Remove from final version. Used for rapid prototyping.
    # train_size, test_size, valid_size = 1000, 100, 100
    # train = Subset(train, torch.randperm(len(train))[:train_size])
    # test = Subset(test, torch.randperm(len(test))[:test_size])
    # valid = Subset(valid, torch.randperm(len(valid))[:valid_size])

    train_loader = DataLoader(
        train, shuffle=True, batch_size=batch_size, collate_fn=collate_fn, num_workers=8
    )
    test_loader = DataLoader(test, shuffle=False, batch_size=batch_size * 2, num_workers=8)
    valid_loader = DataLoader(valid, shuffle=False, batch_size=batch_size * 2, num_workers=8)
    return train_loader, test_loader, valid_loader

## Defining the models

In [23]:
alexnet_model = tv.models.alexnet(weights="DEFAULT")
# Adjusting the last layer to match the number of classes
alexnet_model.classifier[-1] = nn.Linear(
    alexnet_model.classifier[-1].in_features, num_classes
)

resnet34_model = timm.create_model("resnet34", pretrained=True, num_classes=num_classes)
# Addind dropout to the last layer for later fine-tuning
resnet34_model.fc = nn.Sequential(nn.Dropout(0.0), resnet34_model.fc)

vit_model = timm.create_model(
    "vit_small_patch16_224.augreg_in21k_ft_in1k",
    pretrained=True,
    num_classes=num_classes,
)

models = {
    "AlexNet": alexnet_model,
    "ResNet34": resnet34_model,
    "VIT": vit_model,
}

model.safetensors:   0%|          | 0.00/87.3M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/88.2M [00:00<?, ?B/s]

## Defining the training loop

In [24]:
def fit(
    model: nn.Module,
    train_loader: DataLoader,
    valid_loader: DataLoader,
    adam_lr: float = 0.001,
    weight_decay: float = 0.0001,
    epochs: int = 50,
    patience: int = 5,
    bar_postfix: dict[str, str] = {},
) -> tuple[float, list[float], list[float], list[float], list[float]]:
    """
    Fits the model to the data using Adam optimizer and CrossEntropyLoss.

    Parameters:
    - model: The model to be trained.
    - train_loader: The DataLoader for the training set.
    - valid_loader: The DataLoader for the validation set.
    - adam_lr: The learning rate for the Adam optimizer.
    - weight_decay: The weight decay for the Adam optimizer.
    - epochs: The number of epochs to train the model.
    - patience: The number of epochs to wait for the validation loss to improve before stopping the training.
    - bar_postfix: The postfix to be displayed in the progress bars.

    Returns:
    - duration: The duration of the training in seconds.
    - train_loss: The training loss for each epoch.
    - train_acc: The training accuracy for each epoch.
    - valid_loss: The validation loss for each epoch.
    - valid_acc: The validation accuracy for each epoch.
    """
    model.to(device)
    opt = optim.Adam(model.parameters(), lr=adam_lr, weight_decay=weight_decay)
    loss_fn = nn.CrossEntropyLoss()
    multilabel_loss_fn = nn.BCEWithLogitsLoss()  # Required for cutmix augmentations
    epochs_train_loss = []
    epochs_train_acc = []
    epochs_valid_loss = []
    epochs_valid_acc = []

    best_valid_loss = float("inf")
    patience_counter = 0

    start_time = time.time()
    for epoch in range(epochs):
        running_loss = 0.0
        correct, total = 0, 0

        model.train()
        with tqdm(
            train_loader,
            desc=f"Epoch {epoch+1}/{epochs} [TRAIN]",
            leave=False,
            postfix=bar_postfix,
        ) as train_bar:
            for inputs, labels in train_bar:
                inputs, labels = inputs.to(device), labels.to(device)
                predictions = model(inputs)
                if labels.dim() > 1:
                    loss = multilabel_loss_fn(predictions, labels.float())
                else:
                    loss = loss_fn(predictions, labels)
                loss.backward()
                opt.step()
                opt.zero_grad()

                running_loss += loss.item()
                _, classifications = torch.max(predictions, 1)
                if labels.dim() > 1:
                    labels = (labels > 0.35).int()
                    mask = torch.zeros_like(labels).int()
                    mask.scatter_(1, classifications.unsqueeze(1), 1)
                    correct += torch.sum(mask & labels).item()
                else:
                    correct += (classifications == labels).sum().item()
                total += labels.size(0)

        epochs_train_loss.append(running_loss / total)
        epochs_train_acc.append(correct / total)

        model.eval()
        running_loss = 0.0
        correct, total = 0, 0

        with tqdm(
            valid_loader,
            desc=f"Epoch {epoch+1}/{epochs} [VALID]",
            leave=False,
            postfix=bar_postfix,
        ) as valid_bar:
            with torch.no_grad():
                for inputs, labels in valid_bar:
                    inputs, labels = inputs.to(device), labels.to(device)
                    predictions = model(inputs)
                    loss = loss_fn(predictions, labels)

                    running_loss += loss.item()
                    _, classifications = torch.max(predictions, 1)
                    correct += (classifications == labels).sum().item()
                    total += labels.size(0)

        epochs_valid_loss.append(running_loss / total)
        epochs_valid_acc.append(correct / total)

        if epochs_valid_loss[-1] < best_valid_loss:
            best_valid_loss = epochs_valid_loss[-1]
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            break

    end_time = time.time()
    duration = end_time - start_time

    return (
        duration,
        epochs_train_loss,
        epochs_train_acc,
        epochs_valid_loss,
        epochs_valid_acc,
    )

## Defining the evaluation loop

In [25]:
def test(
    model: nn.Module, test_loader: DataLoader, bar_postfix: dict[str, str] = {}
) -> tuple[float, float]:
    """
    Test the model on the test set.

    Parameters:
    - model: The model to be tested.
    - test_loader: The DataLoader for the test set.

    Returns:
    - loss: The loss on the test set.
    - accuracy: The accuracy on the test set.
    """
    model.to(device)
    model.eval()

    loss_fn = nn.CrossEntropyLoss()
    running_loss = 0.0
    correct, total = 0, 0

    with tqdm(
        test_loader, desc=f"[TEST]", leave=False, postfix=bar_postfix
    ) as test_bar:
        with torch.no_grad():
            for inputs, labels in test_bar:
                inputs, labels = inputs.to(device), labels.to(device)
                predictions = model(inputs)
                loss = loss_fn(predictions, labels)

                running_loss += loss.item()
                _, classifications = torch.max(predictions, 1)
                correct += (classifications == labels).sum().item()
                total += labels.size(0)

    loss = running_loss / total
    accuracy = correct / total

    return loss, accuracy

## Simulation loop

In [26]:
def run(
    train_loader: DataLoader,
    test_loader: DataLoader,
    valid_loader: DataLoader,
    lr: float = 0.001,
    weight_decay: float = 0.0001,
    epochs: int = 50,
    patience: int = 5,
    simplify: bool = True,
    bar_extra: dict[str, str] = {},
) -> dict:
    """
    Runs the training and testing on all models.

    Parameters:
    - train_loader: The DataLoader for the training set.
    - test_loader: The DataLoader for the test set.
    - valid_loader: The DataLoader for the validation set.
    - lr: The learning rate for the Adam optimizer.
    - weight_decay: The weight decay for the Adam optimizer.
    - epochs: The number of epochs to train the model.
    - patience: The number of epochs to wait for the validation loss to improve before stopping the training.

    Returns:
    - results: A dictionary containing the results of the training and testing for each model.
    """

    results = {}

    for model_name, model in models.items():
        bar_postfix = {
            "model": model_name,
            "lr": f"{lr}",
            "weight_decay": f"{weight_decay}",
            "patience": f"{patience}",
            "batch_size": f"{train_loader.batch_size}",
            **bar_extra,
        }

        duration, train_loss, train_acc, valid_loss, valid_acc = fit(
            model,
            train_loader,
            valid_loader,
            lr,
            weight_decay,
            epochs,
            patience,
            bar_postfix,
        )
        test_loss, test_acc = test(model, test_loader, bar_postfix)
        results[model_name] = {
            "duration": duration,
            "train_loss": train_loss,
            "train_acc": train_acc,
            "valid_loss": valid_loss,
            "valid_acc": valid_acc,
            "test_loss": test_loss,
            "test_acc": test_acc,
        }

    if simplify:
        for model_name, model_results in results.items():
            results[model_name] = {
                "duration": model_results["duration"],
                "epochs": len(model_results["train_loss"]),
                "train_loss": model_results["train_loss"][-1],
                "train_acc": model_results["train_acc"][-1],
                "valid_loss": model_results["valid_loss"][-1],
                "valid_acc": model_results["valid_acc"][-1],
                "test_loss": model_results["test_loss"],
                "test_acc": model_results["test_acc"],
            }

    return results


def simulate(
    train_loader: DataLoader,
    test_loader: DataLoader,
    valid_loader: DataLoader,
    times: int = 5,
    lr: float = 0.001,
    weight_decay: float = 0.0001,
    epochs: int = 50,
    patience: int = 5,
) -> pd.DataFrame:
    """
    Simulates the training and testing of the models multiple times.

    Parameters:
    - times: The number of times to simulate the training and testing.
    - lr: The learning rate for the Adam optimizer.
    - weight_decay: The weight decay for the Adam optimizer.
    - epochs: The number of epochs to train the model.
    - patience: The number of epochs to wait for the validation loss to improve before stopping the training.

    Returns:
    - df: A DataFrame containing the results.
    """
    results = []
    for i in range(times):
        result = run(
            train_loader,
            test_loader,
            valid_loader,
            lr,
            weight_decay,
            epochs,
            patience,
            bar_extra={"simulation": f"{i+1}/{times}"},
        )
        results.append(result)

    results = [pd.DataFrame.from_dict(result, orient="index") for result in results]
    return pd.concat(results)

## Basic training (no data augmentation)

In [27]:
train_loader, test_loader, valid_loader = get_data(transforms_no_aug)
results = simulate(train_loader, test_loader, valid_loader)
display(results.groupby(results.index).agg(["min", "mean", "max", "std"]))

Epoch 1/50 [TRAIN]:   3%|▎         | 20/739 [00:35<06:48,  1.76it/s, batch_size=256, lr=0.001, model=AlexNet, patience=5, simulation=1/5, weight_decay=0.0001] 

## Data augmentation

In [18]:
transforms_basic_aug = T.Compose(
    [
        T.PILToTensor(),
        T.Resize((224, 224)),  # Default input size for most models
        T.RandomHorizontalFlip(p=0.5),
        T.RandomVerticalFlip(p=0.5),
        T.RandomRotation(degrees=15),
        T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        T.ToDtype(torch.float32, scale=True),
        T.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        ),  # Default values for ImageNet
    ]
)

# CutMix works directly on the batch
cutmix = T.CutMix(num_classes=num_classes)
advanced_transforms = lambda batch: cutmix(*default_collate(batch))

transforms = [
    {
        "transforms": transforms_basic_aug,
        "collate_fn": advanced_transforms,
        "name": "Advanced Augmentations",
    },
    {
        "transforms": transforms_basic_aug,
        "collate_fn": None,
        "name": "Basic Augmentations",
    },
]

results["augmentation"] = "None"
for transform in transforms:
    train_loader, test_loader, valid_loader = get_data(
        transform["transforms"], transform["collate_fn"]
    )
    result = simulate(train_loader, test_loader, valid_loader)
    result["augmentation"] = transform["name"]
    results = pd.concat([results, result])

for model_name, model_results in results.groupby(results.index):
    print(model_name)
    display(model_results.groupby("augmentation").agg(["min", "mean", "max", "std"]))

AlexNet


duration                                   epochs  \
                              min       mean         max        std    min   
augmentation                                                                 
Advanced Augmentations  70.562934  90.638986  127.586732  22.228068     10   
Basic Augmentations     40.306253  62.138282   88.032473  21.071389      6   
None                    12.047885  21.391777   33.380705   8.172049      6   

                                           train_loss            ...  \
                        mean max       std        min      mean  ...   
augmentation                                                     ...   
Advanced Augmentations  12.8  18  3.114482   0.004392  0.004532  ...   
Basic Augmentations      9.2  13  3.114482   0.010050  0.013851  ...   
None                    10.8  17  4.207137   0.000811  0.001466  ...   

                       valid_acc           test_loss                      \
                             max       std       min      mean       max   
augmentation                                                               
Advanced Augmentations      0.47  0.035637  0.013196  0.014086  0.014655   
Basic Augmentations         0.63  0.035777  0.012692  0.015481  0.017392   
None                        0.63  0.040866  0.025636  0.028809  0.032592   

                                 test_acc                         
                             std      min   mean   max       std  
augmentation                                                      
Advanced Augmentations  0.000570     0.47  0.500  0.54  0.027386  
Basic Augmentations     0.001736     0.50  0.526  0.56  0.023022  
None                    0.003354     0.56  0.584  0.60  0.018166  

[3 rows x 32 columns]

ResNet34


duration                                   epochs  \
                              min       mean         max        std    min   
augmentation                                                                 
Advanced Augmentations  49.081903  81.833038  115.891971  31.250895      6   
Basic Augmentations     55.492844  77.543796  110.626211  24.020053      7   
None                    22.045845  33.627481   46.493106   9.383187      7   

                                           train_loss            ...  \
                        mean max       std        min      mean  ...   
augmentation                                                     ...   
Advanced Augmentations  10.0  14  3.807887   0.002699  0.003302  ...   
Basic Augmentations      9.8  14  3.033150   0.001807  0.003922  ...   
None                    10.8  15  3.033150   0.000196  0.000565  ...   

                       valid_acc           test_loss                      \
                             max       std       min      mean       max   
augmentation                                                               
Advanced Augmentations      0.55  0.034351  0.012650  0.014237  0.016329   
Basic Augmentations         0.63  0.037417  0.017251  0.020429  0.026648   
None                        0.68  0.049295  0.014360  0.020801  0.029344   

                                 test_acc                         
                             std      min   mean   max       std  
augmentation                                                      
Advanced Augmentations  0.001535     0.55  0.574  0.61  0.025100  
Basic Augmentations     0.003765     0.51  0.524  0.55  0.016733  
None                    0.005779     0.54  0.594  0.63  0.040988  

[3 rows x 32 columns]

VIT (pretrained)


duration                                    epochs  \
                              min        mean         max        std    min   
augmentation                                                                  
Advanced Augmentations  66.465579  133.461824  171.762095  43.059584      7   
Basic Augmentations     55.323500   83.525748  139.467634  39.623115      6   
None                    26.841427   29.700792   40.630290   6.110977      6   

                                           train_loss            ...  \
                        mean max       std        min      mean  ...   
augmentation                                                     ...   
Advanced Augmentations  14.0  18  4.527693   0.004661  0.004822  ...   
Basic Augmentations      9.0  15  4.242641   0.019884  0.022225  ...   
None                     6.6   9  1.341641   0.002986  0.007029  ...   

                       valid_acc           test_loss                      \
                             max       std       min      mean       max   
augmentation                                                               
Advanced Augmentations      0.34  0.032094  0.019043  0.019564  0.020264   
Basic Augmentations         0.36  0.027019  0.018765  0.020132  0.021019   
None                        0.32  0.039115  0.024996  0.036213  0.043949   

                                 test_acc                         
                             std      min   mean   max       std  
augmentation                                                      
Advanced Augmentations  0.000439     0.24  0.274  0.34  0.038471  
Basic Augmentations     0.001067     0.29  0.316  0.34  0.024083  
None                    0.007206     0.22  0.270  0.30  0.030000  

[3 rows x 32 columns]

## Batch size tuning

In [19]:
batch_sizes = [1, 8, 64]
batch_results = []

for batch_size in batch_sizes:
    train_loader, test_loader, valid_loader = get_data(
        transforms_basic_aug, advanced_transforms, batch_size=batch_size
    )
    result = simulate(train_loader, test_loader, valid_loader)
    result["batch_size"] = batch_size
    batch_results.append(result)

batch_results = pd.concat(batch_results)
for model_name, model_results in batch_results.groupby(batch_results.index):
    print(model_name)
    display(model_results.groupby("batch_size").agg(["min", "mean", "max", "std"]))

AlexNet


duration                                    epochs            \
                   min        mean         max        std    min  mean max   
batch_size                                                                   
1           123.875325  185.086247  275.676433  64.394923      7  10.6  16   
4            76.221509  103.339364  135.745521  22.585316      8  10.8  14   
16           43.404912   71.660870  103.530673  27.955915      6   9.8  14   
32           54.365377   70.603295  111.395533  23.482622      8  10.2  16   
64           47.401097   87.756360  148.540680  44.220029      7  12.6  21   

                     train_loss            ... valid_acc           test_loss  \
                 std        min      mean  ...       max       std       min   
batch_size                                 ...                                 
1           3.781534   0.326447  0.328820  ...      0.11  0.026077  1.151550   
4           2.280351   0.081349  0.081366  ...      0.07  0.000000  0.300353   
16          3.768289   0.020409  0.020420  ...      0.12  0.000000  0.092183   
32          3.346640   0.010393  0.010396  ...      0.13  0.000000  0.046054   
64          6.107373   0.005198  0.005199  ...      0.11  0.000000  0.023029   

                                         test_acc                         
                mean       max       std      min   mean   max       std  
batch_size                                                                
1           1.157232  1.164225  0.004905     0.09  0.098  0.10  0.004472  
4           0.300638  0.301076  0.000293     0.05  0.056  0.08  0.013416  
16          0.092279  0.092381  0.000073     0.10  0.100  0.10  0.000000  
32          0.046072  0.046094  0.000016     0.13  0.130  0.13  0.000000  
64          0.023034  0.023043  0.000005     0.13  0.130  0.13  0.000000  

[5 rows x 32 columns]

ResNet34


duration                                    epochs            \
                   min        mean         max        std    min  mean max   
batch_size                                                                   
1           202.259167  276.692395  372.905331  65.064547      7   9.6  13   
4           110.552607  156.732392  298.352655  79.924237      9  12.6  24   
16          103.096453  117.223044  147.803434  18.488829     12  13.6  17   
32           58.316594  104.230005  162.582873  41.539616      7  12.8  20   
64           63.679890  109.382700  144.020985  35.518718      8  13.6  18   

                     train_loss            ... valid_acc           test_loss  \
                 std        min      mean  ...       max       std       min   
batch_size                                 ...                                 
1           2.302173   0.272748  0.283408  ...      0.17  0.039749  1.150151   
4           6.426508   0.075040  0.075580  ...      0.28  0.013416  0.233386   
16          2.073644   0.018089  0.018344  ...      0.45  0.034641  0.074823   
32          5.167204   0.008927  0.009046  ...      0.36  0.025100  0.033725   
64          4.393177   0.004367  0.004420  ...      0.41  0.035637  0.015196   

                                         test_acc                         
                mean       max       std      min   mean   max       std  
batch_size                                                                
1           1.266129  1.623656  0.200647     0.08  0.156  0.18  0.042778  
4           0.241349  0.249529  0.005835     0.29  0.308  0.32  0.013038  
16          0.079418  0.082679  0.003137     0.33  0.352  0.37  0.014832  
32          0.034795  0.036291  0.000975     0.32  0.342  0.39  0.029496  
64          0.015591  0.016030  0.000340     0.39  0.414  0.46  0.030496  

[5 rows x 32 columns]

VIT (pretrained)


duration                                    epochs           \
                   min        mean         max        std    min mean max   
batch_size                                                                  
1           159.581603  169.361361  197.363984  15.776235      5  5.2   6   
4            64.505119   65.904982   66.718838   0.895811      5  5.0   5   
16           49.367074   49.836792   50.028439   0.273751      5  5.0   5   
32           46.783134   47.870518   48.309932   0.619953      5  5.0   5   
64           46.058612   46.996296   47.799447   0.794427      5  5.0   5   

                     train_loss       ... valid_acc      test_loss           \
                 std        min mean  ...       max  std       min mean max   
batch_size                            ...                                     
1           0.447214        NaN  NaN  ...      0.16  0.0       NaN  NaN NaN   
4           0.000000        NaN  NaN  ...      0.06  0.0       NaN  NaN NaN   
16          0.000000        NaN  NaN  ...      0.09  0.0       NaN  NaN NaN   
32          0.000000        NaN  NaN  ...      0.09  0.0       NaN  NaN NaN   
64          0.000000        NaN  NaN  ...      0.11  0.0       NaN  NaN NaN   

               test_acc                   
           std      min  mean   max  std  
batch_size                                
1          NaN     0.10  0.10  0.10  0.0  
4          NaN     0.13  0.13  0.13  0.0  
16         NaN     0.10  0.10  0.10  0.0  
32         NaN     0.08  0.08  0.08  0.0  
64         NaN     0.12  0.12  0.12  0.0  

[5 rows x 32 columns]

## Learning rate tuning

In [20]:
learning_rates = [0.1, 0.001, 0.00001]
lr_results = []

for lr in learning_rates:
    train_loader, test_loader, valid_loader = get_data(
        transforms_basic_aug, advanced_transforms
    )
    result = simulate(train_loader, test_loader, valid_loader, lr=lr)
    result["lr"] = lr
    lr_results.append(result)

lr_results = pd.concat(lr_results)
for model_name, model_results in lr_results.groupby(lr_results.index):
    print(model_name)
    display(model_results.groupby("lr").agg(["min", "mean", "max", "std"]))

AlexNet


duration                                     epochs            \
               min        mean         max         std    min  mean max   
lr                                                                        
0.00001  41.397519  115.887728  205.680312   70.906592      6  16.8  30   
0.00010  47.994603   63.537085   75.626562   10.367585      7   9.2  11   
0.00100  47.477176   83.655933  110.876495   29.630455      7  12.2  16   
0.01000  41.184559   86.158000  181.598421   54.894500      6  12.4  26   
0.10000  60.882234  119.722955  179.731724   47.857126      9  17.4  26   
1.00000  47.497633  174.100955  339.000342  129.024219      7  25.6  50   

                   train_loss                ... valid_acc            \
               std        min          mean  ...       max       std   
lr                                           ...                       
0.00001  10.329569   0.223931  1.036262e+03  ...      0.15  0.020736   
0.00010   1.483240   0.723341  5.755873e+04  ...      0.08  0.007071   
0.00100   4.324350   0.251809  1.828714e+04  ...      0.11  0.005477   
0.01000   7.829432   0.367419  1.095289e+09  ...      0.10  0.013038   
0.10000   6.804410   0.091545  9.356376e+07  ...      0.16  0.038987   
1.00000  19.086645   0.005212  6.034250e+04  ...      0.14  0.032094   

           test_loss                                           test_acc  \
                 min          mean           max           std      min   
lr                                                                        
0.00001  1469.291563  2.693692e+03  4.313544e+03  1.103902e+03     0.11   
0.00010     6.965146  5.147199e+02  2.545217e+03  1.135083e+03     0.04   
0.00100     7.309915  1.132272e+02  5.118037e+02  2.230585e+02     0.13   
0.01000     5.368096  2.023456e+07  1.011728e+08  4.524584e+07     0.12   
0.10000    15.922511  1.359902e+10  6.658717e+10  2.962755e+10     0.06   
1.00000     0.023275  1.694590e+05  8.472838e+05  3.789156e+05     0.06   

                                
          mean   max       std  
lr                              
0.00001  0.112  0.12  0.004472  
0.00010  0.088  0.10  0.026833  
0.00100  0.130  0.13  0.000000  
0.01000  0.122  0.13  0.004472  
0.10000  0.100  0.15  0.043012  
1.00000  0.092  0.12  0.029496  

[6 rows x 32 columns]

ResNet34


duration                                    epochs            \
               min        mean         max        std    min  mean max   
lr                                                                       
0.00001  48.932917  105.444589  159.650527  41.700871      6  13.2  20   
0.00010  56.757846  112.137844  175.989070  46.641334      7  14.0  22   
0.00100  63.076429   79.757516   97.438762  15.438627      8  10.0  12   
0.01000  48.659825   75.370878  109.858173  29.541853      6   9.4  14   
0.10000  47.842397   79.836834  128.196178  30.382808      6  10.0  16   
1.00000  63.582316   95.094713  119.557237  26.301693      8  12.0  15   

                  train_loss            ... valid_acc           test_loss  \
              std        min      mean  ...       max       std       min   
lr                                      ...                                 
0.00001  5.263079   0.004703  0.004734  ...      0.31  0.015811  0.018650   
0.00010  5.873670   0.004634  0.004697  ...      0.38  0.013038  0.018963   
0.00100  1.870829   0.004692  0.004731  ...      0.34  0.027749  0.017182   
0.01000  3.847077   0.004829  0.004906  ...      0.24  0.043932  0.020460   
0.10000  3.807887   0.005105  0.005180  ...      0.14  0.010954  0.023369   
1.00000  3.316625   0.005318  0.006564  ...      0.15  0.023022  0.030388   

                                            test_acc                         
               mean         max         std      min   mean   max       std  
lr                                                                           
0.00001    0.018793    0.019006    0.000149     0.36  0.380  0.40  0.015811  
0.00010    0.019070    0.019156    0.000074     0.26  0.286  0.32  0.021909  
0.00100    0.017466    0.017623    0.000180     0.23  0.276  0.32  0.032863  
0.01000    0.021421    0.022387    0.000707     0.13  0.166  0.21  0.033615  
0.10000    5.922235   28.798049   12.788468     0.07  0.128  0.15  0.034928  
1.00000  137.809034  589.267305  255.984125     0.08  0.098  0.11  0.013038  

[6 rows x 32 columns]

VIT (pretrained)


duration                                 epochs                \
               min       mean        max       std    min mean max  std   
lr                                                                        
0.00001  46.320469  46.736914  47.107496  0.282528      5  5.0   5  0.0   
0.00010  46.623823  46.893601  47.160277  0.241639      5  5.0   5  0.0   
0.00100  46.000955  46.482002  47.123524  0.442667      5  5.0   5  0.0   
0.01000  46.677069  47.131284  47.615249  0.378577      5  5.0   5  0.0   
0.10000  46.497766  46.913232  47.549822  0.419676      5  5.0   5  0.0   
1.00000  46.267002  46.629714  47.200792  0.373333      5  5.0   5  0.0   

        train_loss       ... valid_acc      test_loss              test_acc  \
               min mean  ...       max  std       min mean max std      min   
lr                       ...                                                  
0.00001        NaN  NaN  ...      0.15  0.0       NaN  NaN NaN NaN     0.12   
0.00010        NaN  NaN  ...      0.13  0.0       NaN  NaN NaN NaN     0.10   
0.00100        NaN  NaN  ...      0.12  0.0       NaN  NaN NaN NaN     0.14   
0.01000        NaN  NaN  ...      0.15  0.0       NaN  NaN NaN NaN     0.06   
0.10000        NaN  NaN  ...      0.08  0.0       NaN  NaN NaN NaN     0.09   
1.00000        NaN  NaN  ...      0.10  0.0       NaN  NaN NaN NaN     0.11   

                          
         mean   max  std  
lr                        
0.00001  0.12  0.12  0.0  
0.00010  0.10  0.10  0.0  
0.00100  0.14  0.14  0.0  
0.01000  0.06  0.06  0.0  
0.10000  0.09  0.09  0.0  
1.00000  0.11  0.11  0.0  

[6 rows x 32 columns]

## Weight decay tuning

In [21]:
weight_decays = [0, 0.001, 0.1]
weight_results = []

for weight_decay in weight_decays:
    train_loader, test_loader, valid_loader = get_data(
        transforms_basic_aug, advanced_transforms
    )
    result = simulate(
        train_loader, test_loader, valid_loader, weight_decay=weight_decay
    )
    result["weight_decay"] = weight_decay
    weight_results.append(result)

weight_results = pd.concat(weight_results)
for model_name, model_results in weight_results.groupby(weight_results.index):
    print(model_name)
    display(model_results.groupby("weight_decay").agg(["min", "mean", "max", "std"]))

KeyboardInterrupt: 

## Dropout tuning

In [ ]:
dropout_probs = [0.0, 0.25, 0.5]
dropout_results = []


def set_dropout(model, dropout_prob):
    for name, module in model.named_modules():
        if isinstance(module, nn.Dropout):
            module.p = dropout_prob
    return model


for dropout_prob in dropout_probs:
    for model_name, model in models.items():
        model = set_dropout(model, dropout_prob)
    train_loader, test_loader, valid_loader = get_data(
        transforms_basic_aug, advanced_transforms
    )
    result = simulate(train_loader, test_loader, valid_loader)
    result["dropout_prob"] = dropout_prob
    dropout_results.append(result)

dropout_results = pd.concat(dropout_results)
for model_name, model_results in dropout_results.groupby(dropout_results.index):
    print(model_name)
    display(model_results.groupby("dropout_prob").agg(["min", "mean", "max", "std"]))

## Ensemble of models

In [ ]:
# TODO: Add ensemble of models